In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 5.4 MB/s 
     |████████████████████████████████| 358 kB 46.4 MB/s 
     |████████████████████████████████| 138 kB 56.3 MB/s 
     |████████████████████████████████| 4.0 MB 28.5 MB/s 
     |████████████████████████████████| 55 kB 4.3 MB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://se

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from collections import deque
import pandas as pd


In [ ]:
def process(sentence,fil):
  return fil.sub(' ',sentence)

def run():
    # Declare variable for chromedriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver',options=options)

    # Declare variable for selenium scraping
    stock_codes={'AAPL':'(Apple|AAPL)','MSFT':'(Microsoft|MSFT)','AMZN':'(Amazon|AMZN)','META':'(Meta|META)','GOOGL':'(Alphabet|Google|GOOGL)','BRK.B':'(Berkshire|BRK.B)','TSLA':'(Tesla|TSLA)','NVDA':'(Nvidia|NVDA)','JPM':'(JPMorgan|JPM)'}
    title = []
    href = []
    tc = []
    mention = []

    # Ask for user input for specific stock news from stock_code list
    print(200*'-')
    sc = input(f'Choose a stock: \n{list(stock_codes.keys())} \n-->')

    # Scrape specific news title and href
    print(200*'-')
    print(f'-->Scraping News for {sc} stock<--\n')
    url = f'https://www.cnbc.com/quotes/{sc}?tab=news'
    driver.get(url)
    try:
      element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "LatestNews-newsTabButton")))
      element.click()

      element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "LatestNews-newsTabButton")))
      element.click()
    except:
      print(f"Cannot find 2nd View More button for {sc}")

    d = driver.page_source
    soup = BeautifulSoup(d,'html.parser')
    group = soup.find_all('div',attrs={'data-analytics':'QuotePage-QuoteNews-1'})
    headlines = deque(group[0].find_all('a'))

    final_headlines = []

    # Check whether it's a legit headline for that news. Video/Club/Pro news are all being removed.
    while headlines:
      temp = headlines.popleft()

      if temp['class'][0] != 'LatestNews-headline':
        headlines.popleft()
      else:
        if not temp.img:
          final_headlines.append(temp)

    for x in final_headlines:
      tc.append(sc)
      mention.append(stock_codes[sc])
      title.append(x['title'])
      href.append(x['href'])
    
    # Declare a dataframe
    df = pd.DataFrame({'Symbol':tc,'Mention':mention,'title':title,'link':href})

    text_list = []

    # Open each href link and read the keypoints text or body texts
    for m,url_new in zip(df['Mention'],df['link']):
      driver.get(url_new)
      filter = re.compile(f'{m}')
      soup_news = BeautifulSoup(driver.page_source,'html.parser')
      keyPoint = soup_news.find_all('div',attrs={'data-test':'keyPoints-1'})

      # If there is no keypoints to be concatenated into sentence
      if not keyPoint:
        body = soup_news.find_all('div',attrs={'data-module':'ArticleBody'})
        group = body[0].find_all('div',attrs={'class':'group'})
        paragraphs = []
        for g in group:
          p_temp = g.find_all('p')
          for p in p_temp: 
            paragraphs.append(p)

        text = ''
        count = 0
        
        # For each paragraph, apply regex filter to find the specific companies mentioned and concatenate the paragraph and next paragraph into sentence
        for p in paragraphs:
          if filter.search(p.text):
            text += p.text.strip()
            count += 1
          if count == 2:
            break

        # If there is no specific companies mentioned in paragraphs. Just choose 1st and 2nd paragraphs and concatenate into sentence
        if not text:
          if len(paragraphs) > 1:
            text = paragraphs[0].text.strip() + paragraphs[1].text.strip()
          else:
            text = paragraphs[0].text.strip()

        text_list.append(text)
    
      # Concatenate all keyspoint into sentence
      else:
        li = keyPoint[0].find_all('li')
        text = ''
        for t in li:
          text += t.text.strip()
        text_list.append(text)

    df['sentence'] = text_list

    # Apply regex for sentence to remove \xa0 symbol
    f = re.compile(f'\xa0')
    df['sentence'] = df['sentence'].apply(process,args=(f,))
    driver.quit()
    print('-->Done<--\n')

    print(df)

In [ ]:
# Run to scrape news for specific stocks.
run()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Choose a stock: 
['AAPL', 'MSFT', 'AMZN', 'META', 'GOOGL', 'BRK.B', 'TSLA', 'NVDA', 'JPM'] 
-->AAPL
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
-->Scraping News for AAPL stock<--

-->Done<--

  Symbol       Mention                                              title  \
0   AAPL  (Apple|AAPL)         Tech leaders react to Roe v. Wade reversal   
1   AAPL  (Apple|AAPL)  Roe v. Wade's demise forces companies to grapp...   
2   AAPL  (Apple|AAPL)  Disney, Apple and Amazon keep waiting as NFL c...   
3   AAPL  (Apple|AAPL)  Apple responds to Roe v. Wade rollback, covers...   
4   AAPL  (Apple|AAPL)  Amazon to invest $23 million to increase 